# Ejercicio 4: Modelo Probabilístico

## Objetivo de la práctica
- Comprender los componentes del modelo vectorial mediante cálculos manuales y observación directa.
- Aplicar el modelo de espacio vectorial con TF-IDF para recuperar documentos relevantes.
- Comparar la recuperación con BM25 frente a TF-IDF.
- Analizar visualmente las diferencias entre los modelos.
- Evaluar si los rankings generados son consistentes con lo que considerarías documentos relevantes.

## Parte 0: Carga del Corpus

In [3]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
corpus = newsgroups.data
target = newsgroups.target
target_names = newsgroups.target_names

## Normalización

In [4]:
import nltk
nltk.download('punkt_tab') # para tokenizar el texto en palabras.
nltk.download('stopwords') # para eliminar palabras vacías como “the”, “and”, “is”, etc.

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wil_s\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wil_s\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


stop_words = set(stopwords.words('english'))

def normalizar(texto):
    texto = texto.lower()  # minúsculas
    texto = re.sub(r'\d+', '', texto)  # eliminar números
    texto = re.sub(r'[^\w\s]', '', texto)  # eliminar puntuación
    tokens = word_tokenize(texto)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    return ' '.join(tokens)

# Aplicar al corpus
corpus_n = [normalizar(doc) for doc in corpus]

## Parte 1: Cálculo de TF, DF, IDF y TF-IDF

### Actividad
1. Utiliza el corpus cargado.
2. Construye la matriz de términos (TF), y calcula la frecuencia de documentos (DF)
3. Calcula TF-IDF utilizando sklearn.
4. Visualiza los valores en un DataFrame para analizar las diferencias entre los términos.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

### Matriz TF, y cálculo de la frecuencia de documentos DF

In [7]:
# Crear matriz de frecuencia de términos
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(corpus_n)
print(f"Shape de la matriz de términos: {X_counts.shape}")

# Obtener nombres de términos
terms = count_vect.get_feature_names_out()

# Convertir a DataFrame (TF)
tf= pd.DataFrame(X_counts.toarray(), columns=terms)
print(tf)

Shape de la matriz de términos: (18846, 122066)
       ___  ____  _____  ______  _______  ________  _________  __________  \
0        0     0      0       0        0         0          0           0   
1        0     0      0       0        0         0          0           0   
2        0     0      0       0        0         0          0           0   
3        0     0      0       0        0         0          0           0   
4        0     0      0       0        0         0          0           0   
...    ...   ...    ...     ...      ...       ...        ...         ...   
18841    0     0      0       0        0         0          0           0   
18842    0     0      0       0        0         0          0           0   
18843    0     0      0       0        0         0          0           0   
18844    0     0      0       0        0         0          0           0   
18845    0     0      0       0        0         0          0           0   

       ___________  _______

In [8]:
# Calcular DF (cuántos documentos contienen cada término)
df_series = (X_counts > 0).sum(axis=0).A1  # A1 convierte a arreglo plano
df= pd.DataFrame({'term': terms, 'df': df_series})
print(df)

           term  df
0           ___  67
1          ____  48
2         _____  42
3        ______  26
4       _______  35
...         ...  ..
122061     zzzs   1
122062   zzzzzz   1
122063  zzzzzzt   1
122064     µsec   1
122065  ÿhooked   1

[122066 rows x 2 columns]


### Calcular TF-IDF

In [9]:
vectorizer = TfidfVectorizer()
corpus_vect = vectorizer.fit_transform(corpus_n)
print(f"Shape del TF-IDF matrix: {corpus_vect.shape}")

Shape del TF-IDF matrix: (18846, 122066)


In [10]:
terms = vectorizer.get_feature_names_out()
tfidf = pd.DataFrame(corpus_vect.toarray(), columns=terms)
print(tfidf)

MemoryError: Unable to allocate 17.1 GiB for an array with shape (18846, 122066) and data type float64

### Visualizar y analizar

In [ ]:
top_df = df.sort_values(by='df', ascending=False).head(10)
print("Top 10 términos por DF:")
print(top_df)


Top 10 términos por DF:
          term    df
118309   would  5259
80782      one  5109
52704     like  4166
26551     dont  3889
50176     know  3784
36854      get  3625
3902      also  3229
108136   think  3127
84169   people  2941
108733    time  2848


In [ ]:
print("TF-IDF del documento 0:")
print(tfidf.iloc[0].sort_values(ascending=False).head(10))


TF-IDF del documento 0:
pens                0.548236
jagr                0.250831
devils              0.216622
bit                 0.193723
fun                 0.185855
regular             0.179938
season              0.173826
nonpittsburghers    0.169476
bashers             0.151134
pulp                0.151134
Name: 0, dtype: float64


## Parte 2: Ranking de documentos usando TF-IDF

### Actividad

1. Dada una consulta, construye el vector de consulta
2. Calcula la similitud coseno entre la consulta y cada documento usando los vectores TF-IDF
3. Genera un ranking de los documentos ordenados por relevancia.
4. Muestra los resultados en una tabla.

## Crear el vector consulta

In [12]:
query = "jersey"

In [13]:
query_vect = vectorizer.transform([query])
print(query_vect)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 122066)>
  Coords	Values
  (0, 47705)	1.0


### Calcular la similitud coseno

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(query_vect, corpus_vect).flatten() # dist arreglo de similitudes de la consulta y cada documento entre [0-1]
print(dist)

[0.10988072 0.         0.         ... 0.         0.         0.        ]


### Generar el ranking y mostrar tabla

In [15]:
top_k = 50

# Ranking de indices debe ser una lista de enteros
ranking_indices = dist.argsort()[::-1] # argsort devuelve índices de menor a mayor y con [::1] invertimos ese orden

# Construcción del DataFrame
resultados = pd.DataFrame({
    'Ranking': range(1, top_k + 1),
    'Documento': [corpus_n[i][:40].replace('\n', ' ') + '...' for i in ranking_indices[:top_k]],
    'Similitud coseno': [dist[i] for i in ranking_indices[:top_k]],
    #'Categoría': [target_names[target[i]] for i in ranking_indices[:top_k]] #
})

print(resultados)

    Ranking                                    Documento  Similitud coseno
0         1                      glassboro new jersey...          0.526563
1         2  help stuck computer new jersey access ra...          0.313241
2         3  thanks replied initial question ive away...          0.287470
3         4  anyone know find modem comm software app...          0.262237
4         5  cds sale jon bon jovi new jersey boomera...          0.261063
5         6  dont forget paul ysebaert exdevil hes go...          0.257928
6         7  boston ottawa first period boston robert...          0.230702
7         8  mention thread selling someones wife guy...          0.208787
8         9  went mess new jersey still waiting refun...          0.208764
9        10  heardperhaps incorrectly lemieux noone w...          0.205492
10       11  toyota camry deluxe sale miles power eve...          0.204255
11       12  ones remember offhand kdka pittsburgh pe...          0.196777
12       13  philadelphia

## Parte 3: Ranking con BM25

### Actividad

1. Implementa un sistema de recuperación usando el modelo BM25.
2. Usa la misma consulta del ejercicio anterior.
3. Calcula el score BM25 para cada documento y genera un ranking.
4. Compara manualmente con el ranking de TF-IDF.

### Libreria BM25

In [21]:
from rank_bm25 import BM25Okapi

# Crear el modelo BM25
bm25 = BM25Okapi(corpus_n)
# Scores BM25 para la consulta
scores_bm25 = bm25.get_scores(query)
# Ranking de índices BM25 
ranking_indices_bm25 = scores_bm25.argsort()[::-1]  # Ordenar de mayor a menor
# Construcción del DataFrame BM25
resultados_bm25 = pd.DataFrame({
    'Documento': [corpus_n[i][:40].replace('\n', ' ') + '...' for i in ranking_indices_bm25[:top_k]],
    'Score BM25': [scores_bm25[i] for i in ranking_indices_bm25[:top_k]],
    'Ranking BM25': range(1, top_k + 1)
})
print(resultados_bm25)


                                      Documento  Score BM25  Ranking BM25
0   cut part mcjsg ajlusxcy dkmetfqxhh mgnm ...    5.316232             1
1   readprint reason joystick stuff take tim...    5.314773             2
2   begin outofcontrolgif mtean___rpn_qjoxqi...    5.313636             3
3   four pseudorandom character generators b...    5.312742             4
4   record shows iisi without cache small en...    5.311135             5
5   try one size rather post name email ill ...    5.290252             6
6   part xrastool cut mfprgkwaqylqjzjhwbbxbn...    5.274772             7
7   cut part m_xnggholizcymwtsva kxfj cabc m...    5.270097             8
8   cut part begin wnexe mko vpivatydm mdyud...    5.269647             9
9   season ticket holder pair giants tickets...    5.257419            10
10  sale jazz compact discs following cds sa...    5.249742            11
11  assume jesuss plea father let cup pass m...    5.247838            12
12  brian guess missed point scale col

## Parte 4: Comparación visual entre TF-IDF y BM25

### Actividad

1. Utiliza un gráfico de barras para visualizar los scores obtenidos por cada documento según TF-IDF y BM25.
2. Compara los rankings visualmente.
3. Identifica: ¿Qué documentos obtienen scores más altos en un modelo que en otro?
4. Sugiere: ¿A qué se podría deber esta diferencia?

## Parte 5: Evaluación con consulta relevante

### Actividad

1. Elige una consulta y define qué documentos del corpus deberían considerarse relevantes.
2. Evalúa Precision@3 o MAP para los rankings generados con TF-IDF y BM25.
3. Responde: ¿Cuál modelo da mejores resultados respecto a tu criterio de relevancia?